## SF Crime Data Analysis with PySpark

In this notebook we use PySpark to explore a dataset from the San Francisco Police Department.
Dataset link: [https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data](https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data)

In [ ]:
# Create a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SF Crime Analysis").getOrCreate()
sc = spark.sparkContext
print('SparkSession started')

In [ ]:
# Read the CSV data using an RDD
from csv import reader

# Read all lines from the CSV file
crime_data_lines = sc.textFile('data/sf_crime.csv')

# Convert each line to a list of strings (removing extra quotes)
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

In [ ]:
# Extract header and filter it out
header = df_crimes.first()
print("Header:", header)
crimes = df_crimes.filter(lambda x: x != header)
print("First two data rows:", crimes.take(2))

In [ ]:
# Convert the RDD of lists to an RDD of Rows
from pyspark.sql import Row

def list_to_row(keys, values):
    row_dict = dict(zip(keys, values))
    return Row(**row_dict)

rdd_rows = crimes.map(lambda x: list_to_row(header, x))
df = spark.createDataFrame(rdd_rows)
print("DataFrame created")

In [ ]:
# Show the DataFrame (first 20 rows)
df.show()

In [ ]:
# Replace columns 'X' and 'Y' with 'Longitude' and 'Latitude' and cast them as float
df = df.withColumn('X', df['X'].cast('float')).withColumn('Y', df['Y'].cast('float'))
df = df.withColumnRenamed('X', 'Longitude').withColumnRenamed('Y', 'Latitude')
print("Columns renamed and cast")

In [ ]:
# Inspect the schema
df.printSchema()

### Exploring the Data

Let's count the number of incidents for each category.

In [ ]:
# Method 1: Using Spark SQL
df.createOrReplaceTempView("crime")
sqlDF = spark.sql("""
    SELECT Category, COUNT(*) AS Count 
    FROM crime 
    GROUP BY Category 
    ORDER BY Count DESC
""")
sqlDF.show(40, False)

In [ ]:
# Method 2: Using DataFrame functions
df.groupBy("Category").count().orderBy("count", ascending=False).show(40, False)

In [ ]:
# Method 3: Using RDD functions
rdd = crimes.map(lambda line: (line[1], 1))
sorted_counts = sorted(rdd.countByKey().items(), key=lambda x: -x[1])
print(sorted_counts)

How about the number of incidents at each district?

In [ ]:
# Method 1: SQL
sqlDF = spark.sql("""
    SELECT PdDistrict, COUNT(*) AS Count 
    FROM crime 
    GROUP BY PdDistrict 
    ORDER BY Count DESC
""")
sqlDF.show()

In [ ]:
# Method 2: DataFrame
df.groupBy("PdDistrict").count().orderBy("count", ascending=False).show()

In [ ]:
# Method 3: RDD functions
rdd_district = crimes.map(lambda line: (line[6], 1))
sorted_district_counts = sorted(rdd_district.countByKey().items(), key=lambda x: -x[1])
print(sorted_district_counts)

Define "downtown" as an area within 0.005 degrees from (37.792489, -122.403221). 
Let's count the number of incidents on Sundays within this area.

In [ ]:
# Method 1: SQL
sqlDF = spark.sql("""
    SELECT Date, DayOfWeek, COUNT(*) AS Count 
    FROM crime 
    WHERE DayOfWeek = 'Sunday'
      AND POW(Latitude - 37.792489, 2) + POW(Longitude + 122.403221, 2) < POW(0.005, 2)
    GROUP BY Date, DayOfWeek 
    ORDER BY Date
""")
sqlDF.show()

In [ ]:
# Method 2: DataFrame
df_downtown = df.filter((df['Latitude'] - 37.792489)**2 + (df['Longitude'] + 122.403221)**2 < 0.005**2)
df_downtown.filter(df_downtown['DayOfWeek'] == 'Sunday') \
    .groupBy("Date", "DayOfWeek") \
    .count() \
    .orderBy('Date') \
    .show()

### Visualizing the Spatial Distribution

Let's make a scatter plot of the crimes.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert the Spark DataFrame to Pandas for plotting
pdf = df.select("Longitude", "Latitude").toPandas()
plt.figure(figsize=(8, 6))
plt.title('SF Crime Distribution')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.scatter(pdf['Longitude'], pdf['Latitude'], s=2, c='r')
plt.show()

### Clustering with Spark ML

Spark ML requires that features be in a single vector column. We'll use the `VectorAssembler` to
combine the `Longitude` and `Latitude` columns into one features column, and then fit a k-means
model (with k=3, chosen arbitrarily).

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

df_coor = df.select("Longitude", "Latitude")
vecAssembler = VectorAssembler(inputCols=["Longitude", "Latitude"], outputCol="features")
new_df = vecAssembler.transform(df_coor).select("features")
new_df.show(10, False)

In [ ]:
# Train a k-means model with k=3 and a fixed seed for reproducibility
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(new_df)

# Print cluster centers
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

# Show cluster memberships
transformed = model.transform(new_df)
transformed.show(20, False)

### Visualizing Clustering Results

You can now visualize the clusters (for example, by converting the predictions back to Pandas
and plotting them with different colors). Below is one example:

In [ ]:
pdf_clusters = transformed.select("features", "prediction").toPandas()
plt.figure(figsize=(8, 6))
plt.title('KMeans Clustering of SF Crimes')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.scatter(pdf_clusters['features'].apply(lambda x: x[0]),
            pdf_clusters['features'].apply(lambda x: x[1]),
            c=pdf_clusters['prediction'], cmap='viridis', s=2)
plt.show()